In [1]:
%load_ext autoreload
%autoreload 2

In [2]:

import os
import pickle

from modules.logging.format_utils import format_measures
from modules.collecting.results_collector import DataFrameCollector
from modules.logging.logger import DefaultLogger
from modules.algorithms.base.OSLPP import Params
from modules.selection.uncertanties import SelectRejectMode
from modules.algorithms.nn.OSLPP_NN_UTILS import train_osda

In [3]:
with open('experiments/configs/small_datasets.pkl', 'rb') as f:
    config = pickle.load(f)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [5]:
# sources = ['art', 'clipart', 'product', 'real_world']
# targets = ['art', 'clipart', 'product', 'real_world']

In [6]:
# sources = ['amazon', 'dslr', 'webcam']
# targets = ['amazon', 'dslr', 'webcam']

In [7]:
# sources = ['painting_train', 'real_train', 'sketch_train']
# targets = ['painting_train', 'real_train', 'sketch_train']

In [8]:
# sources = ['train'] 
# targets = ['validation']

In [9]:
results = DataFrameCollector({'source': [], 'target': [], 'desc': [], 'lr': [], 'seed': [], 'epochs': []})
select_reject_mode = SelectRejectMode.CONFIDENCE
logger = DefaultLogger()
# for source in sources:
#     for target in targets:
#         if source == target:
#             continue
#         for lr in [1e-3, 1e-4, 5e-4]:
#             for n_r in [0.15, 0.25, 0.35, 0.45]:
for (source, target), (common, tgt_private) in config.items():
    for epochs in [10]:
        for lr in [1e-3]:
            for n_r in [0.1, 0.15, 0.25]:
                for seed in range(9):
                    params = Params(pca_dim=512, proj_dim=128, T=10, n_r=n_r, n_r_ratio=None,
                                    dataset='DomainNet_DCC', source=source, target=target,
                                    num_common=len(common), num_src_priv=0, num_tgt_priv=len(tgt_private))
                    metrics = train_osda(params, lr, epochs, select_reject_mode, seed, common, tgt_private, logger)
                    results.collect({'source': source, 'target': target, 'desc': format_measures(metrics), 'lr': lr, 'seed': seed, 'n_r': n_r, 'epochs': epochs})
results.get_result().to_csv('../all_results/results/dcc__conf__small__NEW__variable_epochs__multi_mean_std__nn_raw.csv', header=True, index=False)

painting -> sketch lr= 0.001 seed= 0
[3715 3179  177 ...  489  613  606]
______
Iteration t=1
all:  cs/acc_i=51.81 cs/acc_c=53.33 os/recall_knw=94.87 os/recall_unk=18.39 total/acc_i=38.94 total/acc_c=50.73 total/h_score=27.22
selected:  cs/acc_i=61.89 cs/acc_c=63.35 os/recall_knw=75.32 os/recall_unk=82.89 total/acc_i=66.85 total/acc_c=59.87 total/h_score=68.75
______
Iteration t=2
all:  cs/acc_i=50.49 cs/acc_c=52.10 os/recall_knw=63.47 os/recall_unk=65.58 total/acc_i=48.88 total/acc_c=41.78 total/h_score=50.14
selected:  cs/acc_i=47.67 cs/acc_c=49.71 os/recall_knw=44.40 os/recall_unk=85.86 total/acc_i=52.75 total/acc_c=34.73 total/h_score=46.80
______
Iteration t=3
all:  cs/acc_i=50.49 cs/acc_c=52.09 os/recall_knw=58.00 os/recall_unk=71.86 total/acc_i=49.69 total/acc_c=39.80 total/h_score=49.88
selected:  cs/acc_i=48.36 cs/acc_c=50.36 os/recall_knw=49.22 os/recall_unk=81.03 total/acc_i=50.69 total/acc_c=36.00 total/h_score=47.65
______
Iteration t=4
all:  cs/acc_i=50.23 cs/acc_c=51.78 

KeyboardInterrupt: 